## CNN 

In [ ]:
from PyFunctions import Functions as func
from PyFunctions import var
from PyFunctions import ModelFunc as modelfunc
import os
from tqdm import tqdm
import numpy as np
import cv2

### Images

- Within the folder tests, add whatever image you would like that contains a gun and run the cell below

In [ ]:
#Mobilenet MODEL
edge = False
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
mobilenet = modelfunc.get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MobilenetOld.h5') 

test_folder = 'Tests/Photos'

for i in os.listdir(test_folder):
    if i == 'ipynb_checkpoints': 
        continue
    img_path = f'{test_folder}/{i}'
    predictions = func.get_img_prediction_bounding_box(img_path, mobilenet, dim = var.mobilenet_dimension, edge = edge)
    
    

In [ ]:
#Mobilenet MODEL
edge = False
dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
mobilenet = modelfunc.get_mobilenet(dim)
mobilenet.load_weights('ModelWeights/MV1_NoEdge_NoAugment.h5') 

test_folder = 'Tests/Photos'

for i in os.listdir(test_folder):
    if i == 'ipynb_checkpoints': 
        continue
    img_path = f'{test_folder}/{i}'
    predictions = func.get_img_prediction_bounding_box(img_path, mobilenet, dim = var.mobilenet_dimension, edge = edge, 
                                                      model_type = 'mobilenet')    

In [ ]:
#NORMAL MODEL
edge = False
dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = modelfunc.get_conv_model(dim)
normal_model.load_weights('ModelWeights/V2_NoEdge_NoAugmentation.h5') #path to the model weights

test_folder = 'Tests/Photos'
predictions = []
for idx, i in enumerate([i for i in os.listdir(test_folder) if i != 'ipynb_checkpoints']):
    if idx < 4: 
        continue
    img_path = f'{test_folder}/{i}'
    pred = func.get_img_prediction_bounding_box(img_path, normal_model, dim = var.norm_dimension, edge = edge)
    predictions.append(pred)
    

### Video 
- within the folder Tests, add whatever video (.mp4) that contains a gun and run the cell below
- **Note**: Stick t video shorter than a minute or this process will take a very long time depending on your computer

In [ ]:
edge= False 

model_dim = var.norm_dimension #change this depending on model used
dim = (model_dim[0],model_dim[1], 3)
    
model = modelfunc.get_conv_model(dim)
model.load_weights('ModelWeights/Normal/V2_NoEdge_NoAugmentation.h5')

images = func.get_vid_frames('../Tests/Videos/Pistol2.mp4', model, model_dim, edge = edge, vid_name = 'NormalTest')

In [ ]:
edge= False 

dim = (var.norm_dimension[0],var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('ModelWeights/V2_NoEdge_NoAugmentation.h5')

images = get_vid_frames('../Tests/Videos/Pistol2.mp4', normal_model, var.norm_dimension, edge = edge)

In [ ]:
pbar = tqdm(images, desc= 'Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While')
windows_prob = [window_prob_func(img, normal_model, var.norm_dimension, edge = edge) for img in pbar]

In [ ]:
vid_dim = (224,224)
vid_dim = var.norm_dimension
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Tests/VideoOutput/BoundingBox.mp4',fourcc, 10, vid_dim) #change the filename to whatever you would like


for prob, img in zip(windows_prob, images): 
    clone = img.copy()
    if prob == None:
        out.write(clone)
    else:
        (p, (startx, starty, endx, endy)) = prob
        cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
        out.write(clone)
        
out.release()
cv2.destroyAllWindows()